## Lecture Three: Naive Bayes, Text Classification, and Sentiment
TODO look at the TAD lectures when doing this lecture

**Classification** lies at the heart of both human and machine intelligence.

Many language processing tasks involve classification. In this lecture we'll introduce the naive Bayes algorithm for **text categorization**, the task of assigning a label or category to an entire text or document.

Focus would be on the task of **sentiment analysis**, extraction of **sentiment**, positive or negative orientation that a writer expresses toward some object. e.g. a review of a movie, product or political tweet.

The simplest version of sentiment analysis is binary classification task, and the words of the review provide excellent cues. e.g. words like *great*, *richly*, *awesome*, *pathetic*, *awful* are very informative.

```
+ ...characters and richly applied satire, and some great plot twists
- It was pathetic. The worst part about it was the boxing scenes...
```

**Spam detection** is another important commercial application. Assigning a *spam* or *not-spam* to an email. You could use cues like "WITHOUT ANY COST" or "Dear Winner". It's similar to our blokedin goal.


Another thing we might want to know about the language is in which language it's written in. The task of **language id** is the first step in most language processing pipelines. **Authorship attribution** determines the text's author.

A part-of-speech tagger classifies each occurence of a word, e.g. verb, noun, etc..

The goal of classification is to observe a single observation, extract some useful information and **classify** the observation into one of a discrete set of classes. You could use hand written rules by human but those are fragile. Instead use **supervised machine learning** where we have a data set of input observations, each associated with some correct output. The goal of the algorithm is to learn how to map from a new observation to a correct output.

Formally, the task of classification is to take an input *x* and a fixed set of output classes *Y = {y1, y2, y3, yM}* and return predicted class *y belongs to Y*. 


A **probabilistic classifier** additionally will tell us the probability of the observation being in the class.

**Generative Classifiers** like Naive Bayes build a model of how class could generate some input data; given an observation they return the class most likely to have generated the observation.

**Discriminative Classifiers** like logistic regression instead learn what features from the input are most useful to discriminitate between the different possible classes.


##### Naive Bayes Classifier
We introduce the **multinomial naive Bayes classifier**, it's a bayesian classifier that makes a simplifying (naive) assumption about how features interact.

We represent a document text as if it were a **bag of words**, i.e. an unordered set of words with their position ignored, keeping only their frequency in the document.

Naive Bayes is a probabilistic classifier, meaning that for a document *d*, out of all classes *c belong to C* the classifier returns class *c^* which has the maximum posterior probability given the document. *^* to mean "our correct estimate of the correct class", and we use **argmax** to mean an operation that selects the argument (in this case the class *c*) that maximizes a function (in this case the probability P(c|d)).

![alt text](../images/naive_argmax.png)

This idea's **Bayesian inference**. The intuition of Bayesian inference is to use Bayes' rule to transform equation 4.1 into other probabilities that have some useful properties.

![alt text](../images/bayes_rule.png)


We call Naive Bayes a **generative** model cause we can read eq.4 as stating a kind of implicit assumption about how a document is generated; first a class is sampled from P(c) and then the words are generated by sampling P(d|c).


We compute the most probable class *c^* given some document *d* choosing the class which has the highest product of two probabilities: the **prior probability** of the class P(c) and the **likelihood** of the document *P(d|c)*:

![alt text](../images/prior_likelihood.png)

without loss of generality we can represent a document *d* as a set of features *f1*, *f2*, *fn*:

![alt text](../images/prior_likelihood_features.png)

However, this is still complex to compute cause of the estimation of all the possible proabilities of the combination of the features, e.g. every possible set of words and position. Hence, Naive Bayes make two assumptions:
- bag of words, assuming the features *f1*, *f2*, *fn* encode the word identity and not the position
- **Naive Bayes assumption**: conditional independence assumption that the probabilities *P(fi|c)* are independent given the class *c* and hence can be 'naively' multiplied as follows:

![text alt](../images/naive_bayes_assumption.png)


By considering the features in log space and computing the predicted class as a linear function of input features, classifier that use linear combinations of the inputs to make classification decision like naive bayes and logistic regression are called **linear classifiers**.





Let's implement the Naive Bayes Classifier

Let's first create the **bag of words** from the [IMDB dataset](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews). We'll redo all these lectures in the end with the messages corpus just for fun, for education instead stick to something more serious.







In [ ]:
import pandas as pd

In [ ]:
imdb_dataset_filepath = "../datasets/IMDB.csv"
messages_dataset_filepath = "../datasets/train.csv"

#imdb_dataset = pd.read_csv(imdb_dataset_filepath)
imdb_dataset = pd.read_csv(messages_dataset_filepath)

print(imdb_dataset)
#print(f"total examples={len(imdb_dataset)}")
#print(f"positive examples={len(imdb_dataset[imdb_dataset['sentiment'] == 'positive'])}")
#print(f"negative examples={len(imdb_dataset[imdb_dataset['sentiment'] == 'negative'])}")
#




*P^(c) = Nc/ Ndoc*
so in our case for `positive` it's = 25000 / 50000 = 1/2. Same for `negative`.

In [ ]:
# create the bag of words from the imdb corpus
def get_bag_of_words(dataset):
  bag_of_words = set()
  for row in dataset.iterrows():
    review = row[1]
    review = review.str.split(" ")
    review = review.to_list()[0]
    for word in review:
      bag_of_words.add(word)
  #print(review)
  return bag_of_words

#print(bag_of_words)
#print(len(bag_of_words))
#for word in bag_of_words:
#  print(word)



*P(wi|c)* is the fraction of times the word *wi* appears among all words in all documents of topic c.

We first concatenate all documents with category c into one big “category c” text.

Then we use the frequency of *wi* in this concatenated document to give a maximum likelihood estimate of probability:

![alt text](../images/mle_concatenated_documents.png)


Here the vocabulary *V* consists of all the word types in all classes, not just the words in class *c*.







In [ ]:
documents_of_topic_positive = imdb_dataset[imdb_dataset["block"] == 0]
documents_of_topic_negative = imdb_dataset[imdb_dataset["block"] == 1]






In [ ]:
print(documents_of_topic_negative)

In [ ]:
print(documents_of_topic_positive)

In [ ]:
bag_of_words_in_positive_documents = get_bag_of_words(documents_of_topic_positive)
bag_of_words_in_positive_documents

In [ ]:
bag_of_words_in_negative_documents = get_bag_of_words(documents_of_topic_negative)
bag_of_words_in_negative_documents

Issue with MLE training. Imagine we try to estimate the likelihood of the word "fantastic" given the class "positive", but no training documents that contain the word "fantastic" and is classified as "positive". Perhaps "fantastic" used in a *sarcastic* way in *negative* class. In this case the probability would be 0.

![alt text](../images/sarcastic_probability.png)

Since naive Bayes multiplies all the feature likelihoods together, zero probabilities in the likelihood term for any class will cause the probability of the class to be zero, no matter the other evidence!

The simples solution is to add the Laplace add-one smoothing, this one is commonly used in naive Bayes rather than language models.

![alt text](../images/naive_bayes_add_laplace.png)







In [ ]:
# V is our vocabulary
V = bag_of_words_in_positive_documents.union(bag_of_words_in_negative_documents)
V_sure = get_bag_of_words(imdb_dataset)
assert V == V_sure, f"expected {len(V_sure)} but got {len(V)}"
print(V)

What do we do about **unknown** words? i.e. words that did not occur in the training document of any class and are not in our vocabulary but did appear in the test data.

The solution for this is to ignore them. Remove them from the test document and not include any probability for them at all. TODO why is this the case???


Some systems may choose to ignore another class of words: **stop words**, very frequent words like *the* and *a*. Sort the vocabulary and remove the top 10-100 vocabulary entries as stop words or use already predefined online. Then each istance of these is removed from both test and training documents as if it never occurred.







#### Worked example
let's calculate the *P(c)*

In [ ]:
total_examples = len(imdb_dataset)
positive_example_counts = len(imdb_dataset[imdb_dataset["sentiment"] == "positive"])
negative_example_counts = len(imdb_dataset[imdb_dataset["sentiment"] == "negative"])
prior_positive_class = positive_example_counts / total_examples
prior_negative_class = negative_example_counts / total_examples

print(f"prior_positive_class={prior_positive_class}")
print(f"prior_negative_class={prior_negative_class}")

![alt text](../images/prior_likelihood_features.png)

to apply Naive Bayes classifier to text, we will use each word in the documents as a feature and we consider each of the words in the document by walking an index through every word position in the document.

![alt text](../images/position_words.png)
    

Naive Bayes calculations are done in log space for same reason explained earlier.

![alt text](../images/naive_bayes_log.png)

#### Training the Naive Bayes Classifier
How can we learn the probabilities of *P(c)* i.e. the *prior* and *P(fi|c)* i.e. *likelihood*?

First we'll consider the MLE. Simply use the frequencies in the data. For the class prior *P(c)* we ask which percentage of the documents in our training data class *c* and the *Ndoc* be the total number of documents. Then:













In [ ]:
import math
import numpy as np
import re
from functools import reduce
from collections import Counter


# TODO try this at some point potentially filter stop words
#from sklearn.feature_extraction import text
#stop_words = set(text.ENGLISH_STOP_WORDS)
def get_bag_of_words(dataset):
  return set(
    dataset["text"] # access all the text column
    .dropna()       # remove missing values
    #.str.lower()
    .str.split()    # split on whitespaces (more efficient than " ")
    .explode()      # flatten all words into one Series
    #.loc(lambda x: ~x.isin(stop_words)) # filter stopwords early
  )


# we're keeping the punctuation and everything and just considering the
# words as split by whitespaces so some of these results may not be accurate
def train_naive_bayes():
  total_examples = len(imdb_dataset)
  positive_example_counts = len(imdb_dataset[imdb_dataset["block"] == 0])
  negative_example_counts = len(imdb_dataset[imdb_dataset["block"] == 1])

  prior_positive_class = np.log2(positive_example_counts/total_examples)
  prior_negative_class = np.log2(negative_example_counts/total_examples)

  print(prior_positive_class, prior_negative_class)

  # compute the vocabulary V
  bag_of_words_in_negative_documents = get_bag_of_words(documents_of_topic_negative)
  bag_of_words_in_positive_documents = get_bag_of_words(documents_of_topic_positive)
  V = bag_of_words_in_positive_documents.union(bag_of_words_in_negative_documents)
  V_sure = get_bag_of_words(imdb_dataset)
  assert V == V_sure, f"expected {len(V_sure)} but got {len(V)}"

  bigdoc = {
    "positive": " ".join(documents_of_topic_positive["text"].astype(str)),
    "negative": " ".join(documents_of_topic_negative["text"].astype(str)),
  }

  # precompile a regex pattern to match all words in V
  sorted_words = sorted(V, key=lambda x: (-len(x), x))
  pattern_str = r'\b' + '|'.join([re.escape(word) for word in sorted_words]) + r'\b'
  word_regex = re.compile(pattern_str)

  def count_words(text): return Counter(word_regex.findall(text))

  # do we really need to do it this way???
  # I think we should remove the first 21 entries from these
  positive_counts = count_words(bigdoc["positive"])
  negative_counts = count_words(bigdoc["negative"])

  # build counts dictionary
  counts = {}
  for word in V:
    counts[(word, "positive")] = positive_counts.get(word, 0)
    counts[(word, "negative")] = negative_counts.get(word, 0)

  return V, counts, positive_counts, negative_counts


In [127]:

V, counts, positive_counts, negative_counts = train_naive_bayes()

-2.599037685932879 -0.2602357724811205


In [ ]:
print(V)
print(len(V))

In [ ]:
print(counts)
print(len(counts))

In [128]:
positive_counts

Counter({'to': 29,
         'the': 27,
         'a': 25,
         'and': 22,
         'for': 18,
         'you': 17,
         'is': 17,
         'of': 16,
         'in': 12,
         'with': 12,
         'I': 10,
         'your': 10,
         'are': 10,
         'be': 10,
         'at': 8,
         'Hi': 7,
         'on': 7,
         'have': 6,
         'team': 5,
         'an': 5,
         'thought': 4,
         'can': 4,
         'work': 4,
         'The': 4,
         'Hut': 4,
         'currently': 4,
         'our': 4,
         'if': 4,
         'or': 4,
         'London': 4,
         'role': 4,
         'we': 3,
         'from': 3,
         'connecting': 3,
         'this': 3,
         'as': 3,
         'global': 3,
         'technology': 3,
         'graduates': 3,
         'me': 3,
         'know': 3,
         'up': 3,
         'like': 3,
         'more': 3,
         'interested': 3,
         'experience': 3,
         'will': 3,
         'am': 3,
         'great': 3,
         'd

In [129]:
negative_counts

Counter({'to': 347,
         'a': 329,
         'and': 310,
         'the': 259,
         'you': 185,
         'in': 180,
         'of': 172,
         'for': 143,
         'with': 120,
         'are': 108,
         'be': 98,
         'is': 96,
         'I': 78,
         'at': 78,
         'your': 77,
         'on': 75,
         'have': 65,
         'that': 57,
         '-': 55,
         'this': 54,
         'an': 54,
         'Hi': 49,
         'looking': 47,
         'experience': 46,
         'will': 42,
         'role': 41,
         'working': 41,
         'would': 41,
         'as': 40,
         'our': 39,
         'their': 38,
         'team': 37,
         'from': 36,
         'if': 36,
         'more': 33,
         'can': 33,
         'me': 32,
         'about': 32,
         'new': 32,
         'or': 31,
         'great': 31,
         'they': 30,
         'interested': 29,
         'up': 29,
         'currently': 29,
         '|': 29,
         'am': 28,
         'Developer': 28,


#### Optimizing for Sentiment Analysis

**binary multinomial naive Bayes** or **binary naive Bayes**. For each document we remove all duplicate words before concatenating them into the single big document during training and we also remove duplicates words from the test documents. This is per document binarization so the counts don't have to be binary as it can occur in multiple documents the same word.








#### Naive Bayes for other text classification tasks

**spam detection**



for **language id**



#### Naive Bayes as a Language Model






#### Evaluation: Precision, Recall, F-measure

to evaluate any system for detecting things we build a **confusion matrix**. A confusion matrix is a table for visualizing how an algorithm performs with respect to the human gold labels, using two dimentions (system output, gold labels).

theres a metric called accuracy which asks the question "what percentage of all the observations our system labelled correctly. This is not a great metric for classification as you may and usually have imbalanced classes in the training set e.g. you may have millions of not spam emails and just a few spam emails which then would make it "pointless" to measure the accuracy.

You instead use metrics like **precision** and **recall**. 

- *precision*: number of items that the system deteceted that are in fact positive.

precision = (true positives) / (true positives + false positives)

- *recall*: measures the percentage of items actually present in the input that were correctly identified by the system.

recall = (true positives) / (true positives + false negatives)

there are different metrics that combine precision and recall, one popular one is the **F-measure**. This is the formula for the F-measure:

```
F-measure = (B^2 + 1)PR / (B^2 * P + R)
```

when B is 1, we get the F1 measure:

```
F-measure = 2PR / (P + R)
```

The F-measure comes from the harmonic mean of precision and recall.

**How do you evaluate with more than one class?**

we could compute the precision of a multiclass classification with naive bayes or any other techinique for example when we classify an email into one of these classes "urgent", "spam", "normal" maybe by using **microaveraging** where we compute the performance of each class and average over all classes.

-**macroaveraging**: we compute the performance of each class and average over all classes.
-**microaveraging**: we collect the decision for all classes into a single confusion matrix and then compute precision and recall from that matrix.








#### Test sets and Cross-validation

we use the **training set** to train the model, then we use the **development test set (dev set)** to tune the parameters

A problem having a small data set is that we don't have enough data for training and testing or even when you save a lot of data for trainign and you end up not having enough data for dev/test testing. We can use all of our dataset for training and testing by doing **cross-validation**.

In cross validation we choose a number *k* and partition our data into *k* subsets called **folds**. We choose one of these folds as our test test and train our classifier on the remaining k-1 folds and then compute the error rate on the test set. Then we repeat with another fold until *k* times and average the test error rate from these *k* runs to get an average error rate.

The problem with cross validation is that we're using all of our data for testing so we can't examine any of the data to suggest possible features and influence performance. so what to do? we can create a fixed training set and test set and do the k-fold cross validation on the training set and compute the error rate on the test set as normal.





#### Statistical Significance Testing

how do we know if our system is better than our old one? This is the domain of statistical hypothesis testing.

We'll introduce the statistical significance testing for NLP classifiers.

we have δ delta (greek symbol) that tells us the difference between two classifiers based on a metric *M* on a test dataset *x*.

δ(x) is called the **effect size**. Bigger delta means A is better, small delta means is slighlty better.

Why can we not just check that delta is positive? Cause A can be better than B on that particular test test but is it better on a different dataset?

**null hypothesis** supposes that δ(x) is actually negative or zero, meaning A is not better than B.

the **p-value**. We create a random variable *X* ranging over all test sets and we ask how likely is it, if the null hypothesis was correct, that among these test sets we would encounter the value of δ(x) we found if we repeated the experiment great times?

**p-value**: the probability, assuming the null hypothesis is true, of seeing the δ(x) that we saw or one even greater.

P(δ(X) >= δ(x) | H0 is true)


There are two parameteric tests used in NLP, **approximate randomization** and **bootstrap test**.

**paired tests** are those where we compare two sets of observation that are aligned and each observatino in one set can be paired to an observation in another set.

**Paired Bootstrap Test**

this can be applied to any metric. the term **bootstrapping** means repeatedly drawing large number of samples with replacements (**bootstrap samples**) from an original set. this method makes the assumption that the sample is representative of the population.









#### Avoiding harm in classification

**representational harm** caused by systems that discriminate on social groups, e.g. stereotypes.

**toxiticy detection** is the task of detecting hate speech, abuse, harassement, etc..

false positives could be caused also by bad labelers of course.

release a **model card** with your NLP model. A model card has:
- training algorithm and parameters
- training data sources, motivation, and processing
- evaluation data sources, motivatin and processing
- intended use and users
- model performance across different demographic or other groups
